# Part B

## step 1 - Load the data and add fields to it

### Load the data

In [3]:
import pandas as pd


file_path = "./hotels_data.csv"  
df = pd.read_csv(file_path)


df.head()


,Snapshot ID,Snapshot Date,Checkin Date,Days,Original Price,Discount Price,Discount Code,Available Rooms,Hotel Name,Hotel Stars
0,1,7/17/2015 0:00,8/12/2015 0:00,5,1178,1040,1,6,Best Western Plus Seaport Inn Downtown,3
1,1,7/17/2015 0:00,8/19/2015 0:00,5,1113,982,1,8,Best Western Plus Seaport Inn Downtown,3
2,1,7/17/2015 0:00,8/13/2015 0:00,5,4370,4240,1,3,The Peninsula New York,5
3,1,7/17/2015 0:00,7/26/2015 0:00,5,1739,1667,1,18,Eventi Hotel a Kimpton Hotel,4
4,1,7/17/2015 0:00,8/12/2015 0:00,5,1739,1672,1,3,Eventi Hotel a Kimpton Hotel,4


### Add columns and load to new CSV

In [6]:
#Ensure dates are in correct format
df['Snapshot Date'] = pd.to_datetime(df['Snapshot Date'])
df['Checkin Date'] = pd.to_datetime(df['Checkin Date'])


df['DayDiff'] = (df['Checkin Date'] - df['Snapshot Date']).dt.days
df['WeekDay'] = df['Checkin Date'].dt.day_name()
df['DiscountDiff'] = df['Original Price'] - df['Discount Price']
df['DiscountPerc'] = (df['DiscountDiff'] / df['Original Price']) * 100

output_file_path = "./hotels_data_changed.csv"
df.to_csv(output_file_path, index=False)

print(df.head())

   Snapshot ID Snapshot Date Checkin Date  Days  Original Price  \
0            1    2015-07-17   2015-08-12     5            1178   
1            1    2015-07-17   2015-08-19     5            1113   
2            1    2015-07-17   2015-08-13     5            4370   
3            1    2015-07-17   2015-07-26     5            1739   
4            1    2015-07-17   2015-08-12     5            1739   

   Discount Price  Discount Code  Available Rooms  \
0            1040              1                6   
1             982              1                8   
2            4240              1                3   
3            1667              1               18   
4            1672              1                3   

                               Hotel Name  Hotel Stars  DayDiff    WeekDay  \
0  Best Western Plus Seaport Inn Downtown            3       26  Wednesday   
1  Best Western Plus Seaport Inn Downtown            3       33  Wednesday   
2                  The Peninsula New York   

## step 2 - Best discount code

## step 3 - Clustering Based on price polices 



Identify the 150 hotels with the most data in the dataset and extract their records.

In [ ]:
import pandas as pd

file_path = "./hotels_data.csv"  
df = pd.read_csv(file_path)

hotel_counts = df['Hotel Name'].value_counts()
top_150_hotels = hotel_counts.head(150).index
filtered_df = df[df['Hotel Name'].isin(top_150_hotels)]

filtered_df.head()

,Snapshot ID,Snapshot Date,Checkin Date,Days,Original Price,Discount Price,Discount Code,Available Rooms,Hotel Name,Hotel Stars
2,1,7/17/2015 0:00,8/13/2015 0:00,5,4370,4240,1,3,The Peninsula New York,5
3,1,7/17/2015 0:00,7/26/2015 0:00,5,1739,1667,1,18,Eventi Hotel a Kimpton Hotel,4
4,1,7/17/2015 0:00,8/12/2015 0:00,5,1739,1672,1,3,Eventi Hotel a Kimpton Hotel,4
5,1,7/17/2015 0:00,8/7/2015 0:00,5,1436,1345,1,9,Grand Hyatt New York,4
6,1,7/17/2015 0:00,8/9/2015 0:00,5,1616,1525,1,5,Grand Hyatt New York,4


Find the 40 most common check-in dates  in the dataset and extract their records.


In [16]:
checkin_counts = filtered_df['Checkin Date'].value_counts()
top_40_checkin_dates = checkin_counts.head(40).index
filtered_checkin_df = filtered_df[filtered_df['Checkin Date'].isin(top_40_checkin_dates)]

filtered_checkin_df.head()

,Snapshot ID,Snapshot Date,Checkin Date,Days,Original Price,Discount Price,Discount Code,Available Rooms,Hotel Name,Hotel Stars
2,1,7/17/2015 0:00,8/13/2015 0:00,5,4370,4240,1,3,The Peninsula New York,5
4,1,7/17/2015 0:00,8/12/2015 0:00,5,1739,1672,1,3,Eventi Hotel a Kimpton Hotel,4
7,1,7/17/2015 0:00,8/12/2015 0:00,5,1256,1183,1,5,Grand Hyatt New York,4
8,1,7/17/2015 0:00,8/13/2015 0:00,5,1256,1201,1,54,Grand Hyatt New York,4
29,1,7/17/2015 0:00,8/12/2015 0:00,5,3763,3378,1,39,Park Hyatt New York,5


/var/folders/h9/dr9zhy392ldflkyrxzr5g0gw0000gn/T/ipykernel_96297/3266987204.py:27: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  categorized_df = filtered_checkin_df.groupby('Checkin Date').apply(categorize_prices)


664653

## step 4 - Building a Predictive Model for Hotel Pricing Dynamics Using Snapshot Data



## step 5 - PySpark & Mllib for step 3